In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

In [2]:
scaler = RobustScaler()
impute = SimpleImputer(strategy='most_frequent')

In [3]:
df = pd.read_csv("../data/train.csv", index_col=['Id'])
df.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
Id,,,,,,,,,,,,,,,,,,,,,
000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [139]:
df.Class.value_counts()

Class
0    509
1    108
Name: count, dtype: int64

In [4]:
df = pd.get_dummies(df, columns=['EJ'], dtype=int)
df.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FS,GB,GE,GF,GH,GI,GL,Class,EJ_A,EJ_B
Id,,,,,,,,,,,,,,,,,,,,,
000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1,0,1
007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0,1,0
013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0,0,1
043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0,0,1
044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1,0,1


In [5]:
import torch
from torch import nn, optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [150]:
X = torch.tensor(impute.fit_transform(df.drop(columns=['Class'], axis = 1).values.astype(np.float32)))
y = torch.tensor(df['Class'].to_frame().values.astype(np.float32))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([493, 57]),
 torch.Size([124, 57]),
 torch.Size([493, 1]),
 torch.Size([124, 1]))

In [151]:
from torch.utils.data import TensorDataset, DataLoader

In [152]:
bs = 64
train_set = TensorDataset(X_train, y_train)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=False)
test_set = TensorDataset(X_test, y_test)
testloader = DataLoader(test_set, batch_size=bs, shuffle=False)

In [153]:
from torch import nn

In [154]:
class NNCustom(nn.Module):
   def __init__(self, input_size, output_size, dropout):
      super().__init__()
      self.fc = nn.Sequential(
         nn.Linear(input_size, 128),
         nn.ReLU(),
         nn.Linear(128, 64),
         nn.ReLU(),
         nn.Linear(64, 36),
         nn.ReLU(),
         nn.Linear(36, 16),
         nn.ReLU(),
         nn.Linear(16, output_size),
         nn.Sigmoid()
      )

   def forward(self, x):
      x = self.fc(x)
      return x


In [155]:
config = {
   "input_size": X_train.shape[1],
   "output_size": 1,
   "dropout": 0
}
config

{'input_size': 57, 'output_size': 1, 'dropout': 0}

In [156]:
model = NNCustom(config['input_size'],config['output_size'],config['dropout']).to(device)
criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [157]:
def loop(mode, dataset, dataloader, model, criterion, optimizer, device):
   if mode == "train":
      model.train()
   else:
      model.eval()

   cost = 0
   opt = []
   for feature, target in dataloader:
      feature, target = feature.to(device), target.to(device)
      output = model(feature) # feed forward
      loss = criterion(output, target) # loss

      if mode == "train":
         loss.backward() # backdrop

         optimizer.step() # update weight
         optimizer.zero_grad() # reset optimizer

      cost += loss.item() * feature.shape[0]
      # correct += (output.argmax(1) == target).sum().item()
      opt.append(output)
   cost = cost / len(dataset)
   # acc = correct / len(dataset)
   return cost, opt

In [161]:
trc, tsc = [], []
tro, tso = [], []
i = 0
patience = 0
scr = 5
while True:
   i+=1
   train_cost, train_opt = loop("train", train_set, trainloader, model, criterion, optimizer, device)
   trc.append(train_cost)
   tro.append(train_opt)
   with torch.no_grad():
      test_cost, test_opt = loop("test", test_set, testloader, model, criterion, optimizer, device)
      tsc.append(test_cost)
      tso.append(test_opt)


   print(f"Epoch : {i:4} | train cost : {train_cost:.4f} | test cost : {test_cost:.4f} | patient : {patience}") 
   if train_cost < scr:
      scr = train_cost
      patience = 0
      torch.save(model.state_dict(), "../model/weights.pth")
      log = {
         "train_cost": trc,
         "test_cost": tsc,
      }
      torch.save(log, "../model/logs.pth")
   else:
      patience +=1

   if patience > 5:
      break

Epoch :    1 | train cost : 0.5854 | test cost : 0.5859 | patient : 0
Epoch :    2 | train cost : 0.5786 | test cost : 0.6040 | patient : 0
Epoch :    3 | train cost : 0.5669 | test cost : 0.5745 | patient : 0
Epoch :    4 | train cost : 0.5658 | test cost : 0.5878 | patient : 0
Epoch :    5 | train cost : 0.5627 | test cost : 0.5943 | patient : 0
Epoch :    6 | train cost : 0.5599 | test cost : 0.6170 | patient : 0
Epoch :    7 | train cost : 0.5504 | test cost : 0.6141 | patient : 0
Epoch :    8 | train cost : 0.5467 | test cost : 0.5727 | patient : 0
Epoch :    9 | train cost : 0.5410 | test cost : 0.5306 | patient : 0
Epoch :   10 | train cost : 0.5403 | test cost : 0.5526 | patient : 0
Epoch :   11 | train cost : 0.5434 | test cost : 0.5496 | patient : 0
Epoch :   12 | train cost : 0.5427 | test cost : 0.5923 | patient : 1
Epoch :   13 | train cost : 0.5329 | test cost : 0.5929 | patient : 2
Epoch :   14 | train cost : 0.5249 | test cost : 0.6019 | patient : 0
Epoch :   15 | train

In [160]:
tso

[[tensor([[6.4728e-15],
          [2.2491e-08],
          [1.0356e-12],
          [0.0000e+00],
          [4.6199e-09],
          [1.1564e-04],
          [7.2067e-11],
          [0.0000e+00],
          [6.0717e-04],
          [4.2270e-06],
          [2.7529e-17],
          [1.0418e-01],
          [1.6248e-02],
          [2.0040e-04],
          [4.6124e-05],
          [1.5343e-01],
          [0.0000e+00],
          [2.1864e-03],
          [2.7708e-01],
          [1.1393e-07],
          [4.9200e-01],
          [5.5371e-26],
          [1.6303e-06],
          [1.7545e-06],
          [1.4154e-02],
          [1.4448e-04],
          [1.7911e-06],
          [2.4789e-02],
          [5.7693e-01],
          [1.0151e-10],
          [0.0000e+00],
          [3.1525e-11],
          [3.1790e-03],
          [2.2570e-30],
          [5.0807e-05],
          [0.0000e+00],
          [6.7471e-31],
          [2.1576e-02],
          [4.1104e-06],
          [6.8615e-03],
          [2.0028e-05],
          [4.986

In [111]:
weigth = torch.load("../model/weights.pth", map_location="cpu")
weigth

OrderedDict([('fc.0.weight',
              tensor([[-0.1475, -0.0355, -0.0716,  ..., -0.0353, -0.0448, -0.2032],
                      [ 0.0282,  0.1746, -0.0843,  ..., -0.0232, -0.0597,  0.0611],
                      [ 0.2069,  0.1322,  0.0776,  ..., -0.0516,  0.0153, -0.0486],
                      ...,
                      [ 0.0496,  0.0889,  0.0107,  ...,  0.1323,  0.0264,  0.0140],
                      [ 0.0497,  0.0597,  0.0363,  ..., -0.1528, -0.0199, -0.0225],
                      [ 0.0808,  0.0019,  0.0897,  ...,  0.0831,  0.1607, -0.0385]])),
             ('fc.0.bias',
              tensor([ 0.2007, -0.1234, -0.0307, -0.0194,  0.2114, -0.0074, -0.0774,  0.0569,
                       0.0308,  0.2641, -0.0782,  0.1640,  0.1943, -0.0740,  0.0119,  0.1177,
                       0.0290,  0.2097, -0.1336,  0.0602,  0.2578,  0.0127, -0.1752,  0.0656,
                       0.1162, -0.1149, -0.0238, -0.0098, -0.1507,  0.2153,  0.2671, -0.0600,
                       0.2138,  0.

In [112]:
model.load_state_dict(weigth)
model = model.to(device)

In [113]:
feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)

In [115]:
with torch.no_grad():
   model.eval()
   output = model(feature)
   pred = output.argmax(1)
   acc = (target == pred)
acc

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [116]:
output

tensor([[1.2682e-02],
        [3.1881e-01],
        [8.0296e-02],
        [5.9964e-01],
        [2.4984e-01],
        [5.0532e-02],
        [8.0866e-02],
        [5.6692e-03],
        [6.1591e-01],
        [2.2622e-01],
        [6.6987e-02],
        [2.8958e-01],
        [8.9168e-02],
        [6.9312e-02],
        [3.1122e-01],
        [4.6180e-02],
        [3.0753e-01],
        [4.5866e-01],
        [1.4727e-01],
        [3.2906e-01],
        [1.0824e-02],
        [8.4524e-03],
        [1.4053e-02],
        [1.3840e-02],
        [3.9299e-03],
        [2.3210e-01],
        [3.2662e-04],
        [1.9438e-02],
        [4.0885e-01],
        [4.3483e-03],
        [1.1309e-03],
        [1.6221e-02],
        [7.1825e-04],
        [8.1517e-03],
        [2.7114e-01],
        [2.4487e-01],
        [5.6722e-04],
        [7.2564e-02],
        [4.5177e-01],
        [2.5695e-01],
        [4.2279e-01],
        [1.2276e-01],
        [3.1587e-01],
        [1.0387e-02],
        [5.6755e-01],
        [1